In [19]:
import requests
import json
import pandas as pd
from datetime import datetime, timedelta
import os
import numpy as np
from dotenv import load_dotenv
# !pip install matplotlib
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.express as px
import pytz

load_dotenv()  

True

In [35]:
# Wells List
def get_header_data() -> pd.DataFrame():
    url ='https://api.corva.ai/v1/data/corva/assets'
    args = '?limit=inf&query={county#eq#Alaska}'
    auth = {'Authorization':os.getenv('API_KEY')}

    params = {"sort": '{timestamp: 1}',
              "query": "{county#eq#'Alaska'}",
              "limit":"inf"
            }

    try:
        r = requests.get(url, params=params, headers=auth)
        print(r.url)
        result = json.loads(r.text)
        df = pd.DataFrame.from_records(result)
        df = df.loc[([True if i['name']=='Alaska' else False for i in df['program']])]
        return df
    
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return pd.DataFrame()

    except json.JSONDecodeError as e:
        print(f"Error: {e}")
        return pd.DataFrame()

    except Exception as e:
        print(f"Error: {e}")
        return pd.DataFrame()

def convert_time_range_to_unix_timestamp(start, end, timezone="America/Anchorage", format="%Y-%m-%d %H:%M:%S.%f"):
    local_tz = pytz.timezone(timezone)
    
    dt_start = datetime.strptime(start, format)
    dt_start = local_tz.localize(dt_start)
    dt_start = dt_start.astimezone(pytz.UTC)

    dt_end = datetime.strptime(end, format)
    dt_end = local_tz.localize(dt_end)
    dt_end = dt_end.astimezone(pytz.UTC)

    return int(dt_start.timestamp()), int(dt_end.timestamp())

def get_telemetry_overview_data_by_id(asset_id, hrs=3) -> pd.DataFrame():
    url = f'https://api.corva.ai/v1/data/corva/wits?'
    # url = f'https://api.corva.ai/v1/data/corva/wits.summary-30s?'
    # url = f'https://api.corva.ai/v1/data/corva/wits.summary-1m?'
    auth = {'Authorization':os.getenv('API_KEY')}
    telem_fields ="""timestamp, data.bit_depth, data.block_height, data.hole_depth, data.state"""
    
    # Get well asset ids and df_wits
    df_wits = pd.DataFrame()
    df_headers = get_header_data()
 
    hrs_conv = 3600
    params = {"asset_id": asset_id,
        "sort": '{timestamp: -1}',
        "fields": telem_fields,
        "limit":f"{hrs * hrs_conv}" 
    }

    try:
        # Request corva data
        r = requests.get(url, params=params, headers=auth)
        js = r.json()

        # Append a data to single df_wits
        df_wits = pd.DataFrame.from_records(js)  
        df_wits['time_stamp'] = df_wits['timestamp'].apply(lambda x:datetime.fromtimestamp(x))
        df_wits = pd.concat([df_wits[['timestamp','time_stamp']], pd.DataFrame.from_records(df_wits['data'])], axis=1)
        df_wits['state_drill'] = df_wits['state'].str.contains('Drilling', case=False).astype(int)
        df_wits['bottom_status'] = np.where((df_wits['hole_depth'] - df_wits['bit_depth']) > 190, "off_bottom", "near_bottom")


        # # Add asset, well, rig info to the dataframe
        df_wits['asset_id'] = str(asset_id)
        df_wits['well_name'] = df_headers[df_headers['asset_id'] == asset_id]['name'].to_list()[0].split(' ')[0]
        df_wits['rig_name'] = df_headers[df_headers['asset_id'] ==  asset_id]['rig'].apply(lambda x: x['name']).to_list()[0]
        
        # Make sure the timestamp is a DateTimeIndex
        df_wits.set_index('time_stamp', inplace=True)
        
        # Resample to 10-second intervals 
        # todo: not sure if this is the best way to do this
        df_wits = df_wits.resample('10S').agg(
            {col: 'mean' if df_wits[col].dtype in ['float64', 'int64'] else lambda x: x.value_counts().index[0] for col in df_wits.columns})
        df_wits = df_wits.reset_index()
        
    except requests.exceptions.RequestException as e:
        print(f"Error 1: {e}")

    except json.JSONDecodeError as e:
        print(f"Error 2: {e}")

    except Exception as e:
        print(f"Error 3: {e}")

    return df_wits

def get_telemetry_data_by_id_date_range(asset_id, start, end) -> pd.DataFrame():
    url = f'https://api.corva.ai/v1/data/corva/wits?'
    auth = {'Authorization':os.getenv('API_KEY')}
    telem_fields ="""timestamp, data.bit_depth, data.block_height, data.hole_depth, 
    data.hook_load, data.rotary_rpm, data.rotary_torque, data.tvd,
    data.pump_spm_total, data.standpipe_pressure, data.mud_flow_in, data.mud_flow_in, 
    data.rop, data.weight_on_bit, data.mwd_annulus_ecd, data.mud_flow_out_percent, 
    data.state, data.rigtime, data.mud_density
    """
    
    df_wits = pd.DataFrame()
    
    # Get well asset ids
    df_headers = get_header_data()
    
   # Format timestampes to unix int ranges
    unix_start_ts, unix_end_ts = convert_time_range_to_unix_timestamp(start=start, end=end)
    
    params = {"asset_id": asset_id,
        "sort": '{timestamp: -1}',
        "fields": telem_fields,
        "limit":"inf",
        "query": '{timestamp#gte#'+str(unix_start_ts)+ '}AND{timestamp#lte#'+str(unix_end_ts)+'}'
    }

    try:
        # Request corva data
        r = requests.get(url, params=params, headers=auth)
        js = r.json()
        
        # Append a data to single df_wits
        df_wits = pd.DataFrame.from_records(js)  
        df_wits['time_stamp'] = df_wits['timestamp'].apply(lambda x:datetime.fromtimestamp(x))
        df_wits = pd.concat([df_wits[['timestamp','time_stamp']], pd.DataFrame.from_records(df_wits['data'])], axis=1)
       
        df_wits['state_drill'] = df_wits['state'].str.contains('Drilling', case=False).astype(int)
        df_wits['bottom_status'] = np.where((df_wits['hole_depth'] - df_wits['bit_depth']) > 190, "off_bottom", "near_bottom")

        # Add asset, well, rig info to the dataframe
        df_wits['asset_id'] = str(asset_id)
        df_wits['well_name'] = df_headers[df_headers['asset_id'] == asset_id]['name'].to_list()[0].split(' ')[0]
        df_wits['rig_name'] = df_headers[df_headers['asset_id'] ==  asset_id]['rig'].apply(lambda x: x['name']).to_list()[0]
        # print(df_wits.columns)
    except requests.exceptions.RequestException as e:
        print(f"Error 1: {e}")

    except json.JSONDecodeError as e:
        print(f"Error 2: {e}")

    except Exception as e:
        print(f"Error 3: {e}")

    return df_wits


In [70]:
fig = px.line(df, x='time_stamp', y='bh_deriv', title='Block Height Derivative vs Time')
fig.update_layout(xaxis_title='Time', yaxis_title='Block Height Derivative')

# Set threshold value
threshold = df['threshold'].max()

# Add threshold line
fig.add_trace(
    go.Scatter(
        x=df['time_stamp'],
        y=[threshold]*len(df),
        mode='lines',
        name='threshold',
        line=dict(color='red', width=2, dash='dash')
    )
)

fig.write_html("file.html", auto_open=True)


In [43]:

pd.set_option('display.max_rows', 5000)
df_orig = get_telemetry_overview_data_by_id(88220415,6)  #64689114 88220415
df_orig['bh_deriv'] = df_orig['block_height'].diff()

df = df_orig[df_orig['bottom_status'] == "near_bottom"]

# Assume 'time_stamp' is the datetime index
df.set_index('time_stamp', inplace=True)

# Resample to 15-minute windows and get max value
df_resampled = df['bh_deriv'].resample('30S').max()
threshold = df_resampled.std() * 7.5
df_resampled = df_resampled.reset_index()
df_resampled.columns = ['time_stamp', 'bh_deriv']
df_resampled['threshold'] = threshold
df_resampled['is_Conn'] = df_resampled['bh_deriv'].where(df_resampled['bh_deriv'] > threshold)

df_resampled = df_resampled[df_resampled['is_Conn'].notna()]
df_resampled['std_num'] = df_resampled.reset_index().index +1

df_conns = df_resampled[['bh_deriv','std_num']]

df_out = df_orig.merge(df_conns, left_on='bh_deriv', right_on='bh_deriv', how='left')
df_out['std_num'] = df_out['std_num'].fillna(method='ffill').fillna(0).astype(int)

df_out

https://api.corva.ai/v1/data/corva/assets?sort=%7Btimestamp%3A+1%7D&query=%7Bcounty%23eq%23%27Alaska%27%7D&limit=inf


,time_stamp,timestamp,bit_depth,block_height,hole_depth,state,state_drill,bottom_status,asset_id,well_name,rig_name,bh_deriv,std_num
0,2023-07-01 13:04:40,1.688245e+09,19054.040000,63.890000,19054.040000,Rotary Drilling,1,near_bottom,88220415,MT7-14,Doyon 25,NaN,0
1,2023-07-01 13:04:50,1.688245e+09,19054.282000,63.630000,19054.282000,Rotary Drilling,1,near_bottom,88220415,MT7-14,Doyon 25,-0.260000,0
2,2023-07-01 13:05:00,1.688246e+09,19054.732000,63.178000,19054.732000,Rotary Drilling,1,near_bottom,88220415,MT7-14,Doyon 25,-0.452000,0
3,2023-07-01 13:05:10,1.688246e+09,19055.181000,62.735000,19055.181000,Rotary Drilling,1,near_bottom,88220415,MT7-14,Doyon 25,-0.443000,0
4,2023-07-01 13:05:20,1.688246e+09,19055.648000,62.249000,19055.648000,Rotary Drilling,1,near_bottom,88220415,MT7-14,Doyon 25,-0.486000,0
5,2023-07-01 13:05:30,1.688246e+09,19056.116000,61.782000,19056.116000,Rotary Drilling,1,near_bottom,88220415,MT7-14,Doyon 25,-0.467000,0
6,2023-07-01 13:05:40,1.688246e+09,19056.601000,61.302000,19056.601000,Rotary Drilling,1,near_bottom,88220415,MT7-14,Doyon 25,-0.480000,0
7,2023-07-01 13:05:50,1.688246e+09,19057.081000,60.827000,19057.081000,Rotary Drilling,1,near_bottom,88220415,MT7-14,Doyon 25,-0.475000,0
8,2023-07-01 13:06:00,1.688246e+09,19057.567000,60.353000,19057.567000,Rotary Drilling,1,near_bottom,88220415,MT7-14,Doyon 25,-0.474000,0
9,2023-07-01 13:06:10,1.688246e+09,19058.041000,59.878000,19058.041000,Rotary Drilling,1,near_bottom,88220415,MT7-14,Doyon 25,-0.475000,0
